# 如何追踪大语言模型的 token 使用量

追踪 [token](/docs/concepts/tokens/) 使用量以计算成本，是将应用投入生产的重要环节。本指南将介绍如何从 LangChain 模型调用中获取此信息。

:::info 先决条件

本指南假定您熟悉以下概念：

- [大语言模型](/docs/concepts/text_llms)
:::

## 使用 LangSmith

您可以使用 [LangSmith](https://www.langchain.com/langsmith) 来帮助追踪大语言模型应用中的 token 使用量。请参阅 [LangSmith 入门指南](https://docs.smith.langchain.com/)。

## 使用回调

有一些 API 特定的回调上下文管理器，允许您跨多次调用追踪 token 使用量。您需要检查您的特定模型是否支持此类集成。

如果您的模型不支持此类集成，您可以创建一个自定义回调管理器，方法是借鉴 [OpenAI 回调管理器](https://python.langchain.com/api_reference/community/callbacks/langchain_community.callbacks.openai_info.OpenAICallbackHandler.html) 的实现。

### OpenAI

我们首先来看一个追踪单一聊天模型调用的 token 使用量的极其简单的示例。

:::danger

回调处理器目前不支持旧版语言模型（例如 `langchain_openai.OpenAI`）的流式 token 计数。有关在流式处理上下文中的支持，请参阅此处关于聊天模型的相应指南：[/docs/how_to/chat_token_usage_tracking](/docs/how_to/chat_token_usage_tracking)。

:::

### 单次调用

In [1]:
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

with get_openai_callback() as cb:
    result = llm.invoke("Tell me a joke")
    print(result)
    print("---")
print()

print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")



Why don't scientists trust atoms?

Because they make up everything.
---

Total Tokens: 18
Prompt Tokens: 4
Completion Tokens: 14
Total Cost (USD): $3.4e-05


### 多次调用

上下文管理器内的所有内容都将被跟踪。下面是一个示例，展示了如何使用它来跟踪链式调用中的多次调用。这也适用于可能使用多个步骤的 Agent。

In [2]:
from langchain_community.callbacks import get_openai_callback
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

template = PromptTemplate.from_template("Tell me a joke about {topic}")
chain = template | llm

with get_openai_callback() as cb:
    response = chain.invoke({"topic": "birds"})
    print(response)
    response = chain.invoke({"topic": "fish"})
    print("--")
    print(response)


print()
print("---")
print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")



Why did the chicken go to the seance?

To talk to the other side of the road!
--


Why did the fish need a lawyer?

Because it got caught in a net!

---
Total Tokens: 50
Prompt Tokens: 12
Completion Tokens: 38
Total Cost (USD): $9.400000000000001e-05


## 流式输出

:::danger

`get_openai_callback` 目前不支持流式输出的 token 计数（例如 `langchain_openai.OpenAI` 等旧版语言模型）。如果您希望在流式输出的上下文中正确计算 token 数量，有以下几种选择：

- 按照[此指南](/docs/how_to/chat_token_usage_tracking)中的说明使用聊天模型；
- 实现一个[自定义回调处理器](/docs/how_to/custom_callbacks/)，该处理器使用适当的 token 分割器来计算 token；
- 使用 LangSmith 等监控平台[LangSmith](https://www.langchain.com/langsmith)。
:::

请注意，在使用旧版语言模型进行流式输出时，token 计数不会更新：

In [3]:
from langchain_community.callbacks import get_openai_callback
from langchain_openai import OpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

with get_openai_callback() as cb:
    for chunk in llm.stream("Tell me a joke"):
        print(chunk, end="", flush=True)
    print(result)
    print("---")
print()

print(f"Total Tokens: {cb.total_tokens}")
print(f"Prompt Tokens: {cb.prompt_tokens}")
print(f"Completion Tokens: {cb.completion_tokens}")
print(f"Total Cost (USD): ${cb.total_cost}")



Why don't scientists trust atoms?

Because they make up everything!

Why don't scientists trust atoms?

Because they make up everything.
---

Total Tokens: 0
Prompt Tokens: 0
Completion Tokens: 0
Total Cost (USD): $0.0
